In [1]:
import gw_ml_priors

In [2]:
import multiprocessing
import os
import shutil
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bilby.core.prior import Interped
from joblib import Parallel, delayed
from numpy.random import uniform as unif
from tqdm.auto import tqdm

from gw_ml_priors.conversions import calc_a2
from gw_ml_priors.regressors.scikit_regressor import ScikitRegressor

NUM_CORES = multiprocessing.cpu_count()


def get_a1_prior(xeff, q, mcmc_n=int(1e5)):
    a1s = np.linspace(0, 1, 500)
    da1 = a1s[1] - a1s[0]
    p_a1 = Parallel(n_jobs=NUM_CORES, verbose=1)(
        delayed(get_p_a1_given_xeff_q)(a1, xeff, q, mcmc_n)
        for a1 in tqdm(a1s, desc="Building a1 cache")
    )
    p_a1 = p_a1 / np.sum(p_a1) / da1
    data = pd.DataFrame(dict(a1=a1s, p_a1=p_a1))
    a1 = data.a1.values
    p_a1 = norm_values(data.p_a1.values, a1)
    min_b, max_b = find_boundary(a1, p_a1)
    return Interped(
        xx=a1, yy=p_a1, minimum=min_b, maximum=max_b, name="a_1", latex_label=r"$a_1$"
    )


def get_p_a1_given_xeff_q(a1, xeff, q, n=int(1e4)):
    cos1, cos2 = unif(-1, 1, n), unif(-1, 1, n)
    a2 = calc_a2(xeff=xeff, q=q, cos1=cos1, cos2=cos2, a1=a1)
    integrand = a2_interpreter_function(a2)
    return np.mean(integrand)


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def find_boundary_idx(x):
    """finds idx where data is non zero (assumes that there wont be gaps)"""
    non_z = np.nonzero(x)[0]
    return non_z[0], non_z[-1]


def norm_values(y, x):
    return y / np.trapz(y, x)


def find_boundary(x, y):
    b1, b2 = find_boundary_idx(y)
    vals = [x[b1], x[b2]]
    start, end = min(vals), max(vals)
    return start, end


def a2_interpreter_function(a2):
    return np.where(((0 < a2) & (a2 < 1)), 1, 0)




In [3]:

# outdir = "out_p_a1_given_q_xeff"
# os.makedirs(outdir, exist_ok=True)
# q, xeff = 0.5, -0.4
# a1_prior = get_a1_prior(q=q, xeff=xeff)
# plt.plot(a1_prior.xx, a1_prior.yy)
# plt.xlabel("a1")
# plt.ylabel(f"p(a1|q={q},xeff={xeff})")

## TODO: train ML model to generate p(a1|xeff,q) for different q, xeff

In [21]:
# q_range = np.linspace(0, 1, 11)    # q range 0 to 1
# xeff_range = np.linspace(-1, 1, 11)  # xeff range -1 to 1
# print(q_range)
# print(xeff_range)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[-1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8  1. ]


In [5]:
q_range = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
xeff_range = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
print(q_range)
print(xeff_range)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


### Generate data and save

In [7]:
outdir = "out_pisitive_xeff"
os.makedirs(outdir, exist_ok=True)

In [9]:
data = []
for i in q_range:
    for j in xeff_range:
        print(f"q={i}, xeff={j}")
        a1_prior = get_a1_prior(q=i, xeff=j)
        plt.plot(a1_prior.xx, a1_prior.yy)
        plt.xlabel("a1")
        plt.ylabel(f"p(a1|q,xeff)")
        plt.title(f"q={i}, xeff={j}")
        plt.savefig(f"{outdir}/p_a1_given_q_xeff_{i}_{j}.png")
        plt.close()
        # save to data
        data.append([i,j,a1_prior.xx,a1_prior.yy])
        



q=0.1, xeff=0.1


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    7.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   15.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   29.9s
Building a1 cache: 100%|██████████| 500/500 [00:32<00:00, 15.31it/s]
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   33.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


q=0.1, xeff=0.2


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   10.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   25.7s
Building a1 cache: 100%|██████████| 500/500 [00:28<00:00, 17.53it/s]
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   29.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


q=0.1, xeff=0.3


...

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   10.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   24.4s
Building a1 cache: 100%|██████████| 500/500 [00:27<00:00, 18.46it/s]
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   28.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


q=1.0, xeff=0.8


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   10.5s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   23.2s
Building a1 cache: 100%|██████████| 500/500 [00:25<00:00, 19.43it/s]
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   26.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


q=1.0, xeff=0.9


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    9.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   21.9s
Building a1 cache: 100%|██████████| 500/500 [00:24<00:00, 20.47it/s]
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:   25.2s finished


In [10]:
df = pd.DataFrame(data)

In [11]:
df.columns = ["q","xeff","a1","p_a1"]

In [12]:
import pickle
# save the dataframe to pickle
with open(f"{outdir}/p_a1_positive_xeff.pkl", "wb") as f:
    pickle.dump(df, f)

In [13]:
df

,q,xeff,a1,p_a1
0,0.1,0.1,"[0.012024048096192383, 0.014003959823454523, 0...","[0.009216418726310445, 0.028458441503637077, 0..."
1,0.1,0.2,"[0.1222444889779559, 0.12400351805816039, 0.12...","[0.0013008215791223796, 0.004041149522956928, ..."
2,0.1,0.3,"[0.23246492985971942, 0.23400307629286626, 0.2...","[0.0010697856954987733, 0.0024382824268816272,..."
3,0.1,0.4,"[0.3426853707414829, 0.3440026345275721, 0.345...","[0.0009629691381983512, 0.00222891654232283, 0..."
4,0.1,0.5,"[0.45290581162324645, 0.454002192762278, 0.455...","[0.001315515963205737, 0.0027549382395791115, ..."
...,...,...,...,...
85,1.0,0.5,"[0.002004008016032064, 0.004003999983935807, 0...","[0.004842086019793161, 0.00951858515585301, 0...."
86,1.0,0.6,"[0.20240480961923846, 0.2040031967743101, 0.20...","[0.0010434286955042496, 0.0013208399318507351,..."
87,1.0,0.7,"[0.40280561122244485, 0.4040023935646844, 0.40...","[0.0018007422164230593, 0.0023384387900644075,..."
88,1.0,0.8,"[0.6032064128256512, 0.6040015903550587, 0.604...","[0.0032252396366210193, 0.004504994041532742, ..."


Some data contains nans, remove them so that they won't cause problems later.

In [14]:
# test pickle file
out = pd.read_pickle(f"{outdir}/p_a1_positive_xeff.pkl")

In [15]:
out

,q,xeff,a1,p_a1
0,0.1,0.1,"[0.012024048096192383, 0.014003959823454523, 0...","[0.009216418726310445, 0.028458441503637077, 0..."
1,0.1,0.2,"[0.1222444889779559, 0.12400351805816039, 0.12...","[0.0013008215791223796, 0.004041149522956928, ..."
2,0.1,0.3,"[0.23246492985971942, 0.23400307629286626, 0.2...","[0.0010697856954987733, 0.0024382824268816272,..."
3,0.1,0.4,"[0.3426853707414829, 0.3440026345275721, 0.345...","[0.0009629691381983512, 0.00222891654232283, 0..."
4,0.1,0.5,"[0.45290581162324645, 0.454002192762278, 0.455...","[0.001315515963205737, 0.0027549382395791115, ..."
...,...,...,...,...
85,1.0,0.5,"[0.002004008016032064, 0.004003999983935807, 0...","[0.004842086019793161, 0.00951858515585301, 0...."
86,1.0,0.6,"[0.20240480961923846, 0.2040031967743101, 0.20...","[0.0010434286955042496, 0.0013208399318507351,..."
87,1.0,0.7,"[0.40280561122244485, 0.4040023935646844, 0.40...","[0.0018007422164230593, 0.0023384387900644075,..."
88,1.0,0.8,"[0.6032064128256512, 0.6040015903550587, 0.604...","[0.0032252396366210193, 0.004504994041532742, ..."
